### 1. Парсинг новостей

Для парсинга новостей был выбран портал Lenta.ru, особенностью которого является наличие динамической кнопки "Лента добра", отфильтровывающей негативные новости. 

Для работы с динамическим элементом использовалась библиотека `Selenium`. В вызове функции предусмотрена возможность как включать, так и выключать кнопку через параметр `good_news=True`. Основной сбор данных со страницы выполняется с помощью библиотеки `BeautifulSoup`. Количество статей, которое необходимо собрать, задаётся через параметр `num_articles` функции `parse_page`.

In [1]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import pandas as pd
import requests

def get_info(good_news=True, html='https://lenta.ru/parts/news/'):
    """ 
    This function retrieves information from the Lenta.ru newsfeed, with a custom option to 'show only good news'.
    """
    if good_news==True:  
        s = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=s)
        driver.get(html)
        driver.implicitly_wait(3)
        driver.find_element(by=By.CLASS_NAME, value="goodnews").click()
        soup = BeautifulSoup(driver.page_source, 'html.parser') 
        
    else:
        html = requests.get('https://lenta.ru/parts/news/', verify=False)
        soup = BeautifulSoup(html.text, 'html.parser') 
    
    urls = soup.find_all('a', class_="card-full-news _parts-news")
    dict_url = {i: urls[i].get("href") for i in range(len(urls))}
    headers = soup.find_all('h3', class_="card-full-news__title")
    dict_header = {i: headers[i].get_text() for i in range(len(headers))}

    final_df = pd.DataFrame(columns=['url', 'header', 'article'])
    final_df['url'] = pd.DataFrame.from_dict(dict_url, orient='index', columns=['url'])
    final_df['header'] = pd.DataFrame.from_dict(dict_header, orient='index', columns=['header'])


    dict_article = {}
    for i in range(len(dict_url)):
        url = 'https://lenta.ru' + dict_url[i]
        response = requests.get(url)
        soup2 = BeautifulSoup(response.text, 'html.parser') 
        text = soup2.find_all('p', class_="topic-body__content-text")
        article_list = []
        for t in text:
            article = article_list.append(t.get_text())
            article = ' '.join(article_list)
        dict_article[i] = article

    final_df['article'] = pd.DataFrame.from_dict(dict_article, orient='index', columns=['article'])
    return final_df

def parse_page(good_news=True, num_articles=25):
    """ 
    The function iterates over the pages of the newsfeed to retrieve the number of articles specified in the function call.
    """
    df_final = pd.DataFrame(columns=['url', 'header', 'article'])
    
    for i in range(1,1000):
        if df_final.shape[0] < num_articles:
            n = num_articles-df_final.shape[0]
            temp_df = get_info(good_news, html='https://lenta.ru/parts/news/' + str(i))
            df_final = pd.concat([df_final, temp_df.iloc[:n]], ignore_index=True)

        else:
            return df_final

In [2]:
data = parse_page(good_news=True, num_articles=50)
data

,url,header,article
0,/news/2025/07/11/kultovyy/,«Декстер» вернулся с новым сезоном и воскресши...,На Paramount+ начал выходить сериал «Декстер: ...
1,/news/2025/07/10/rossiyanka-uvidela-medvezhat-...,"Россиянка увидела медвежат, захотела угостить ...","Россиянка увидела медвежат, захотела угостить ..."
2,/news/2025/07/10/rossiyskiy-region-vvedet-novy...,Российский регион введет новые правила для тур...,Северная Осетия начала разрабатывать новые пра...
3,/news/2025/07/10/dagestan-postroit-dorogu-k-sa...,Дагестан построит дорогу к самому большому в Е...,В Дагестане построят дорогу к Сарыкумскому пес...
4,/news/2025/07/10/pochti-million-turistov-poset...,Почти миллион туристов посетили Кабардино-Балк...,Почти миллион туристов посетили Кабардино-Балк...
5,/news/2025/07/10/v-rossii-poyavilsya-putevodit...,В России появился путеводитель по промышленном...,В России презентовали путеводитель по промышле...
6,/news/2025/07/10/nazvan-samyy-poseschaemyy-gor...,Назван самый посещаемый город Золотого кольца ...,Ярославль оказался самым посещаемым городом Зо...
7,/news/2025/07/10/pervaya-sumka-herm-s-birkin-u...,Первая сумка Hermès Birkin ушла на торгах за 8...,Первый прототип сумки Hermès Birkin продали за...
8,/news/2025/07/10/rossiyskoe-vino-priznali-luch...,Российское вино признали лучшим на конкурсе в ...,Российское вино признали лучшим на международн...
9,/news/2025/07/10/v-vologde-nashli-zahoronenie-...,В Вологде нашли захоронение второго главы города,В Вологде строители нашли захоронение второго ...


### 2. Суммаризация и расчет метрик

Для суммаризации текста была выбрана модель `sarahai/ruT5-base-summarizer`, основанная на архитектуре `ai-forever/ruT5-base`(разработка команды SberDevices). Для обучения модели авторами был использован датасет с 60 тысячами строк. В работе использовалась уже обученная нейронная сеть с фиксированными параметрами.

Генерация суммирующего абзаца строилась со следующими ограничениями: минимальный размер - `80` токенов, максимальный размер - `250` токенов. Устанавливая параметр `length_penalty=1.0`, мы поощряем модель давать более длинные (и, как правило, более содержательные в контексте задачи) ответы. Параметр `num_beams=4` означает, что на каждом шаге генерации рассматривается 4 наиболее вероятных вариантов продолжения последовательности. Флажок `early_stopping=False` означает, что генерация продолжается, пока есть хотя бы один потенциальный кандидат.

Для оценки суммаризации были выбраны две метрики из библиотеки `evaluate` - `rouge` и `bertscore`.

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from evaluate import load

model_name = "sarahai/ruT5-base-summarizer"  
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

rouge = load("rouge")
bertscore = load("bertscore")

for i in range(len(data)):
    input_ids = tokenizer(data['article'].values[i], return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=250, min_length=80, length_penalty=1.0, num_beams=4, early_stopping=False)
    data.loc[i, 'summary'] = tokenizer.decode(outputs[0], skip_special_tokens=True)

    rouge_metrics = rouge.compute(predictions=[data.loc[i, 'summary']], references=[data.loc[i, 'article']])

    data.loc[i, 'rouge1'] = rouge_metrics['rouge1']
    
    bertscore_metrics = bertscore.compute(predictions=[data.loc[i, 'summary']], references=[data.loc[i, 'article']], lang="ru")
    
    data.loc[i, 'precision'] = bertscore_metrics['precision']
    data.loc[i, 'recall'] = bertscore_metrics['recall']
    data.loc[i, 'f1'] = bertscore_metrics['f1']

### 3. Результаты

In [10]:
data

,url,header,article,summary,rouge1,precision,recall,f1
0,/news/2025/07/11/kultovyy/,«Декстер» вернулся с новым сезоном и воскресши...,На Paramount+ начал выходить сериал «Декстер: ...,На стриминговом сервисе Paramount+ начался пок...,0.296296,0.728377,0.612097,0.665193
1,/news/2025/07/10/rossiyanka-uvidela-medvezhat-...,"Россиянка увидела медвежат, захотела угостить ...","Россиянка увидела медвежат, захотела угостить ...","Россиянка, увидев медвежат, захотела угостить ...",0.000000,0.828361,0.646386,0.726147
2,/news/2025/07/10/rossiyskiy-region-vvedet-novy...,Российский регион введет новые правила для тур...,Северная Осетия начала разрабатывать новые пра...,Северная Осетия начала разрабатывать новые пра...,0.000000,0.916484,0.748953,0.824292
3,/news/2025/07/10/dagestan-postroit-dorogu-k-sa...,Дагестан построит дорогу к самому большому в Е...,В Дагестане построят дорогу к Сарыкумскому пес...,В Дагестане построят дорогу к Сарыкумскому пес...,0.000000,0.951297,0.776635,0.855138
4,/news/2025/07/10/pochti-million-turistov-poset...,Почти миллион туристов посетили Кабардино-Балк...,Почти миллион туристов посетили Кабардино-Балк...,Почти миллион туристов посетили Кабардино-Балк...,1.000000,0.871038,0.731394,0.795132
5,/news/2025/07/10/v-rossii-poyavilsya-putevodit...,В России появился путеводитель по промышленном...,В России презентовали путеводитель по промышле...,В России презентовали путеводитель по промышле...,0.000000,0.872907,0.723139,0.790996
6,/news/2025/07/10/nazvan-samyy-poseschaemyy-gor...,Назван самый посещаемый город Золотого кольца ...,Ярославль оказался самым посещаемым городом Зо...,Ярославль оказался самым посещаемым городом Зо...,0.571429,0.853228,0.704813,0.771952
7,/news/2025/07/10/pervaya-sumka-herm-s-birkin-u...,Первая сумка Hermès Birkin ушла на торгах за 8...,Первый прототип сумки Hermès Birkin продали за...,Первый прототип сумки Herms Birkin продали за ...,0.480000,0.915247,0.756415,0.828285
8,/news/2025/07/10/rossiyskoe-vino-priznali-luch...,Российское вино признали лучшим на конкурсе в ...,Российское вино признали лучшим на международн...,Российское вино признано лучшим на международн...,0.521739,0.873984,0.684426,0.767677
9,/news/2025/07/10/v-vologde-nashli-zahoronenie-...,В Вологде нашли захоронение второго главы города,В Вологде строители нашли захоронение второго ...,В Вологде строители нашли захоронение второго ...,0.400000,0.913074,0.736089,0.815085


In [11]:
data[['rouge1', 'precision', 'recall', 'f1']].mean()

rouge1       0.402223
precision    0.878469
recall       0.720545
f1           0.791524
dtype: float64

Среди доступных rouge-метрик нами была выбрана `rouge1`, которая измеряет перекрытие токенов (униграмм) в нашем оригинальном тексте и сгенерированном резюме. Недостаток этой метрики состоит в том, что изначально она предназначена для сравнения суммаризаций - произведенной человеком (эталонной) и сгенерированной моделью. В нашем варианте идеальный вариант суммаризации отсутствует, что искажает интерпретацию результатов. Так как это числовая метрика, которая просто считает отношение количества токенов в одном и другом тексте, то для большинства из статей ее значения очень малы. Например, первая статья состоит из более чем 1 тысячи токенов, тогда как для суммаризации мы установили ограничение в 250 токенов - естественным образом, мы получаем малое значение `rouge1 = 0.2963`. 

В то же время высокие показатели этой метрики мы можем наблюдать на коротких статьях, что означает, что сгенерированная суммаризация действительно состоит из слов оригинального текста. Однако нужно отметить, что метрика не принимает во внимание смысловое соответствие, а лишь дает нам возможность оценить количество совпадений слов.

Вторая метрика - `bertscore` - также оценивает сходство токенов в оригинальном и сгенерированном текстах, но принимает во внимание семантическое сходство, рассчитывая косинусное подобие между токенами.
Bertscore дает нам возможность оценить три показателя:

- `precision` - насколько сгенерированный текст соответствует содержанию оригинальной статьи;

- `recall` - насколько оригинальная статья отражена в сгенерированном тексте. Эта метрика способна оценить, были ли **упущены важные детали** из оригинальной статьи;

- `f1 score` - взвешенная метрика между precision и recall, позволяющая дать суммаризации обобщенную оценку.

Полученные результаты демонстрируют достаточно высокий уровень `precision` (среднее - 0.8785), что свидетельствует о том, что сгенерированные суммаризации в целом соответствуют содержанию оригинальных текстов. При этом значения `recall` оказываются заметно ниже (среднее - 0.7205), что может указывать на потерю части важной информации из исходных статей. Повышение обоих показателей может рассматриваться как одна из задач для дальнейшего улучшения модели суммаризации.

В процессе вычитки были обнаружены следующие **ошибки модели суммаризации**:

1) Дублирование предложений: 

*Россиянка, увидев медвежат, захотела угостить их колбаской и сняла все это на видео. Женщина обратилась к водителю автомобиля с просьбой поделиться лакомством с дикими животными. В итоге один из «медвежаток» встает на задние лапы, а другой — встает на задние лапы. В то же время один из «медвежаток» встает на задние лапы.*

*Пять фасонов популярной в 1980-х годах обуви вернулись в моду летом 2025 года. Так, актуальными вновь стали двухцветные лодочки, лодочки и лодочки. Кроме того, число трендов пополнили балетки, балетки, туфли с открытой пяткой и оксфорды. Кроме того, число трендов пополнили балетки, туфли с открытой пяткой и модели с Т-образным ремешком.*

2) Галлюцинации:

- Семантические: 

*Официальный представитель МИД России Мария Захарова раскритиковала рубашку министра иностранных дел России Сергея Лаврова на гала-ужине в Куала-Лумпуре.* (Оригинальный текст не содержит оценочного суждения внешнего вида политика).

*Актриса китайского происхождения Фэйя Цзоу, выросшая в России, поделилась мнением о Владивостоке и его жителях. По словам Цзоу, она не была никогда во Владивостоке, но очень хотела бы приехать в город, чтобы познакомиться с ним. По словам Цзоу, она занимает первое место в России по изучению китайского языка, обгоняя даже Москву и Санкт-Петербург.* (Первое место в России по изучению китайского языка занимает не девушка, а город).

- Фактические: 

*В России презентовали путеводитель по промышленному туризму. В нем туристы смогут увидеть искры доменной печи.* (В путеводителе нельзя увидеть искры, но можно узнать, где их увидеть).

*Редакторы испанской версии журнала G раскрыли пять самых модных трендов 2026 года*. (Оригинальное название журнала - GQ).


### 4. Обсуждение и дальнейшие пути изучения

В ходе выполнения задания рассматривались и другие варианты **оценки качества суммаризации**. В частности, одним из возможных подходов может стать оценка резюме по установленным параметрам (например, точность, полнота, галлюцинации) **с использованием генеративной нейросети**. Однако при тестировании этого решения были выявлены следующие проблемы:

- не все доступные генеративные модели корректно работают с русскоязычными текстами;
- большинство бесплатных моделей имеет ограничение на количество обрабатываемых токенов;
- использование модели для оценки результатов другой модели может приводить к накоплению ошибок и снижению достоверности итоговой оценки.

Кроме того, в нашем кейсе для решения задачи суммаризации была использована уже обученная модель с заранее настроенными весами. Однако в перспективе возможно не только более детальное сравнение с другими доступными моделями, но и **проведение собственного дообучения или обучения с нуля** на специально подобранном корпусе текстов. Это позволит адаптировать модель к специфике задач и языкового материала, а также потенциально повысить качество суммаризации.

### Ссылки

Gaby A. (2025). T5-Base, T5-Large, and BART — The Battle of the Summarization Models. Medium. Ссылка: https://medium.com/@gabya06/t5-vs-bart-the-battle-of-the-summarization-models-c1e6d37e56ca

Prabhat Zade (2025). ROUGE Score: A Complete Tutorial for Evaluating Text Summarization Models. Medium. Ссылка: https://medium.com/@prabhatzade/rouge-score-a-complete-tutorial-for-evaluating-text-summarization-models-a3a146417118

ruT5-base. Hugging Face. Ссылка: https://huggingface.co/ai-forever/ruT5-base

ruT5-base-summarizer. Hugging Face. Ссылка: https://huggingface.co/sarahai/ruT5-base-summarizer

Tianyi Zhang, Varsha Kishore, Felix Wu, Kilian Q. Weinberger, and Yoav Artzi. (2020). BERTScore: Evaluating Text Generation with BERT. Конференция ICLR, 2020. Ссылка: https://arxiv.org/abs/1904.09675

Transformers — Text Generation. Hugging Face Documentation. Ссылка: https://huggingface.co/docs/transformers/main_classes/text_generation